In [1]:
from gensim.models.ldamulticore import LdaMulticore
import pandas as pd


In [2]:
from gensim import corpora

In [3]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import spacy
import csv
import email
nlp = spacy.load("en_core_web_sm")


In [4]:
stop_words = set(stopwords.words('english'))
def token_of_email(email):
    lemmas = nlp(email)
    # add NER
    return [w.lemma_ for w in lemmas if not w.lemma_.lower() in stop_words and w.lemma_.isalpha()]


In [5]:
import pickle

In [6]:
with open("pickled_dict_all", "rb") as pick:
    dictionary = pickle.load(pick) 

In [7]:
id2word = corpora.Dictionary([dictionary])

In [8]:
paths = pd.read_csv("paths.csv")["path"].to_list()

In [9]:
import time

In [10]:
class MyCorpus():
    def __iter__(self):
        for path in paths:
            with open(path, "r", encoding="windows-1252") as file:
                sample = email.message_from_file(file)
            message = sample._payload
            token = token_of_email(message)
            yield id2word.doc2bow(token)

In [11]:
corpus = MyCorpus()

In [12]:
nlp.max_length = 2*1e7

In [ ]:
tic = time.perf_counter()
lda = LdaMulticore(corpus, id2word=id2word, num_topics=250, workers=11)
toc = time.per_counter
print(f"time {toc-tic}")

In [ ]:
with open("model_lda_full", "ab") as pick:
    pickle.dump(lda, pick) 